# Customer Churn Prediction: Data Cleaning
In this notebook, we will focus on cleaning the customer churn dataset to ensure it is ready for further analysis and modeling.
## Key Steps in this Notebook:
- Handle missing data for both numerical and categorical columns.
- Correct data types to ensure each feature that the appropriate data type.
- Detect and handle outliers in numerical features.
- Remove any duplicate records to avoid data redundancy.

### Load Libraries and Dataset

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Load the dataset
try:
    df = pd.read_csv("../data/raw/Telco-Customer-Churn.csv")
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print(f"File not found.")

# Display the first few rows of the dataset
df.head()

Dataset loaded successfully!


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Identify Missing Data

In [14]:
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_data = pd.DataFrame({"Missing_values":missing_values,"Percentage":missing_percentage})
missing_data
"""
If more than zero we can sort those by percentage
missing_data = missing_data[missing_data["Missing_values"]>0].sort_values(by="Percentage",ascending=False)
"""

,Missing_values,Percentage
customerID,0,0.0
gender,0,0.0
SeniorCitizen,0,0.0
Partner,0,0.0
Dependents,0,0.0
tenure,0,0.0
PhoneService,0,0.0
MultipleLines,0,0.0
InternetService,0,0.0
OnlineSecurity,0,0.0


### Handle Missing Data
Although we do not have any missing values in out dataset, we will still show the implementation of how we could handle missing values if we had some.

#### Numeric Columns
For numerical columns, we will fill missing values with the median since it is more robust to outliers than the mean.


In [23]:
# Covert "TotalCharges" to numeric
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
print(df["TotalCharges"].dtype)

# We will assume this columns has missing value
# Fill missing values in "TotalCharges" with the median
df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)
print(df["TotalCharges"].isnull().sum())

float64
0


C:\Users\dell\AppData\Local\Temp\ipykernel_12884\1143301255.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)
